In [1]:
import os.path as op
import gffutils

# http://bacteria.ensembl.org/info/website/ftp/index.html
# ftp://ftp.ensemblgenomes.org/pub/release-32/bacteria//gtf/bacteria_0_collection/mycobacterium_tuberculosis_h37rv/

fn = op.expanduser('~/Dropbox/data/ensembl/Mycobacterium_tuberculosis_h37rv.ASM19595v2.32.gtf.gz')
db_name = op.expanduser('~/Dropbox/data/ensembl/Mycobacterium_tuberculosis_h37rv.ASM19595v2.32.db')

In [6]:
#db = gffutils.create_db(fn, dbfn=db_name, force=True, keep_order=True, merge_strategy='merge', sort_attribute_values=True)

In [7]:
db = gffutils.FeatureDB(db_name, keep_order=True)

In [67]:
def extract_annotations(db):
    '''
    Extract gene annotations from the gff db
    
    Paramters
    ---------
    db: gffutils.FeatureDB
        The database containing annotations database
    
    Returns:
    --------
    gene_annotations: [{'gene_name': ...}]
        Annotations containing the gene starts, ends and exons
    '''
    annotations = []

    for gene in db.features_of_type('gene', order_by='start'):
        #print("gene:", gene)
        transcripts = db.children(gene, featuretype='transcript')
        transcript_objs = []

        gene_type = ''

        try:
            gene_type = gene['gene_biotype']
        except Exception as ex:
            pass

        for transcript in transcripts:
            #print("children:", list(db.children(transcript)))

            try:
                start_codon = next(db.children(transcript, featuretype='start_codon'))
                end_codon = next(db.children(transcript, featuretype='stop_codon'))

                transcript_obj = {
                    'start_codon': { 'start': start_codon.start, 
                                    'end': start_codon.end,
                                   'chrom': start_codon.chrom,
                                   },
                    'end_codon': { 'start': end_codon.start, 
                                  'end': end_codon.end,
                                 'chrom': end_codon.chrom,
                                 }
                    }
            except Exception as ex:
                pass

            exons = db.children(transcript, featuretype='exon')
            exon_objs = []
            for exon in exons:
                exon_objs += [{
                    'start': exon.start,
                    'end': exon.end,
                    'chrom': exon.chrom,
                }]
            transcript_obj['exons'] = exon_objs

            cdss = db.children(transcript, featuretype='CDS')
            cds_objs = []
            for cds in cdss:
                cds_objs += [{
                    'start': cds.start,
                    'end': cds.end,
                    'chrom': cds.chrom,
                }]
            transcript_obj['CDSs'] = cds_objs

            transcript_objs += [transcript_obj]

        gene_name = ''

        try:
            gene_name = gene['gene_name'][0]
        except Exception as ex:
            pass

        try:
            gene_id = gene['gene_id'][0]
        except Exception as ex:
            pass

        annotations += [{
            'start': gene.start,
            'end': gene.end,
            'chrom': gene.chrom,
            'gene_name': gene_name,
            'gene_type': gene_type,
            'ensembl_id': gene_id,
            'transcripts': transcript_objs,  
        }]

    return annotations

annotations = extract_annotations(db)

for a in annotations[:50]:
    print(a['ensembl_id'], a['gene_name'], a['gene_type'], a['start'])
#print([a['id'] for a in annotations[:10]])

Rv0001 dnaA ['protein_coding'] 1
Rv0002 dnaN ['protein_coding'] 2052
Rv0003 recF ['protein_coding'] 3280
Rv0004  ['protein_coding'] 4434
Rv0005 gyrB ['protein_coding'] 5240
Rv0006 gyrA ['protein_coding'] 7302
Rv0007  ['protein_coding'] 9914
EBG00000313329 ileT ['tRNA'] 10887
EBG00000313365 alaT ['tRNA'] 11112
Rv0008c  ['protein_coding'] 11874
Rv0009 ppiA ['protein_coding'] 12468
Rv0010c  ['protein_coding'] 13133
Rv0011c  ['protein_coding'] 13714
Rv0012  ['protein_coding'] 14089
Rv0013 trpG ['protein_coding'] 14914
Rv0014c pknB ['protein_coding'] 15590
Rv0015c pknA ['protein_coding'] 17467
Rv0016c pbpA ['protein_coding'] 18759
Rv0017c rodA ['protein_coding'] 20231
Rv0018c pstP ['protein_coding'] 21637
Rv0019c fhaB ['protein_coding'] 23270
Rv0020c fhaA ['protein_coding'] 23861
EBG00000313373 leuT ['tRNA'] 25644
Rv0021c  ['protein_coding'] 25913
Rv0022c whiB5 ['protein_coding'] 27023
Rv0023  ['protein_coding'] 27595
Rv0024  ['protein_coding'] 28362
Rv0025  ['protein_coding'] 29245
Rv0026 

In [68]:
annotations[:5]

[{'chrom': 'Chromosome',
  'end': 1524,
  'ensembl_id': 'Rv0001',
  'gene_name': 'dnaA',
  'gene_type': ['protein_coding'],
  'start': 1,
  'transcripts': [{'CDSs': [{'chrom': 'Chromosome', 'end': 1521, 'start': 1}],
    'end_codon': {'chrom': 'Chromosome', 'end': 1524, 'start': 1522},
    'exons': [{'chrom': 'Chromosome', 'end': 1524, 'start': 1}],
    'start_codon': {'chrom': 'Chromosome', 'end': 3, 'start': 1}}]},
 {'chrom': 'Chromosome',
  'end': 3260,
  'ensembl_id': 'Rv0002',
  'gene_name': 'dnaN',
  'gene_type': ['protein_coding'],
  'start': 2052,
  'transcripts': [{'CDSs': [{'chrom': 'Chromosome',
      'end': 3257,
      'start': 2052}],
    'end_codon': {'chrom': 'Chromosome', 'end': 3260, 'start': 3258},
    'exons': [{'chrom': 'Chromosome', 'end': 3260, 'start': 2052}],
    'start_codon': {'chrom': 'Chromosome', 'end': 2054, 'start': 2052}}]},
 {'chrom': 'Chromosome',
  'end': 4437,
  'ensembl_id': 'Rv0003',
  'gene_name': 'recF',
  'gene_type': ['protein_coding'],
  'star